In [6]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import pandas as pd

# import the data and the sam data for cleaning
df1 = pd.read_csv("./code_enforcement_violations.csv")
df2 = pd.read_csv("./Live_Street_Address_Management_(SAM)_Addresses.csv")
df3 = pd.read_csv("./build_prop_violations.csv")
df1

C:\Users\maste.DESKTOP-6HD7KEF\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8,9,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,case_no,ticket_no,status_dttm,status,code,value,description,violation_stno,violation_sthigh,violation_street,...,ward,contact_addr1,contact_addr2,contact_city,contact_state,contact_zip,sam_id,latitude,longitude,location
0,CE737959,BAA1QM02,2024-02-03 17:14:00+00,Open,1,25,Improper storage trash: res,2,NaN,Union Park,...,3,2 UNION PK ST,NaN,BOSTON,MA,02118,139372,42.341980,-71.070151,"(42.34197953680218, -71.07015050989436)"
1,CE737958,LEHCWL02,2024-02-03 16:40:00+00,Open,1,25,Improper storage trash: res,46,NaN,Newton,...,9,447 SHAWMUT AV,C/O LAILA MAALOUF,BOSTON,MA,02118,102642,42.339440,-71.074631,"(42.3394395428274, -71.07463052618378)"
2,CE737957,DUVKL802,2024-02-03 16:15:00+00,Open,1,25,Improper storage trash: res,294,296,Columbus,...,4,296 COLUMBUS AV,NaN,BOSTON,MA,02116,163593,42.346545,-71.073958,"(42.346545424034154, -71.07395778109957)"
3,CE737956,2L5NDF16,2024-02-03 16:14:00+00,Open,3,100,Overfilling of barrel/dumpster,504,NaN,Dudley,...,8,520 DUDLEY ST,NaN,ROXBURY,MA,02119,226709,42.322790,-71.072701,"(42.322789542194485, -71.07270055853654)"
4,CE737954,SZSDFX08,2024-02-03 16:03:00+00,Open,9a,250,Illegal dumping < 1 cubic yd,58,NaN,Mozart,...,10,23 WINDSOR DR,NaN,FOXBORO,MA,02035,99205,42.320800,-71.104401,"(42.32079958526445, -71.10440063469422)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683258,CE217197,CQV206,2000-01-01 18:54:00+00,Closed,1,25,Improper storage trash: res,73,NaN,Worcester,...,9,73 WORCESTER ST,NaN,BOSTON,MA,02118-3904,152520,42.339420,-71.077691,"(42.33941954680697, -71.07769053299197)"
683259,CE217196,CP0Q06,2000-01-01 18:51:00+00,Closed,1,25,Improper storage trash: res,79,NaN,Worcester,...,9,79 WORCESTER ST,NaN,BOSTON,MA,02118,152532,42.339550,-71.077851,"(42.33954954765962, -71.07785053236364)"
683260,CE217195,CKAU06,2000-01-01 18:44:00+00,Closed,1,25,Improper storage trash: res,92,NaN,Worcester,...,9,92 WORCESTER ST,C/O ROBERT WHALEN,BOSTON,MA,02118,152569,42.339640,-71.078541,"(42.33963954823288, -71.07854053395985)"
683261,CE217194,CDE806,2000-01-01 18:33:00+00,Closed,1,25,Improper storage trash: res,17,NaN,Worcester,...,9,C/O URBAN MANAGEMENT,35 FAY ST,BOSTON,MA,02118,152404,42.338020,-71.076031,"(42.338019545006, -71.07603053224193)"


In [11]:
# rename the columns in the sam dataset to prepare for merging
df2.rename(columns={'SAM_ADDRESS_ID': 'sam_id', 'FULL_ADDRESS': 'full_address'}, inplace=True)
# select only the wanted columns
df2 = df2[['full_address', 'sam_id']]
# fix sam_id in df3 to get rid of the decimal, just in case
# we drop missing sam_id for all data
df3.dropna(subset = ['sam_id'], inplace = True)
df1.dropna(subset = ['sam_id'], inplace = True)
df3['sam_id'] = df3['sam_id'].astype(int)

In [23]:
# merge the datasets
merged_code_vio = pd.merge(df1, df2, on=['sam_id'], how='left')
merged_build_vio = pd.merge(df3, df2, on=['sam_id'], how='left')

In [24]:
# drop the empty addresses, caused by missing sam addresses from what I can tell
merged_code_vio.dropna(subset = ['full_address'], inplace = True)
merged_build_vio.dropna(subset = ['full_address'], inplace = True)

In [25]:
# final clean by dropping apartment numbers ex. 13 Commonwealth #1 -> 13 Commonwealth
merged_code_vio['full_address'] = merged_code_vio['full_address'].str.split(' #')
merged_build_vio['full_address'] = merged_build_vio['full_address'].str.split(' #')
merged_code_vio

,case_no,ticket_no,status_dttm,status,code,value,description,violation_stno,violation_sthigh,violation_street,...,contact_addr1,contact_addr2,contact_city,contact_state,contact_zip,sam_id,latitude,longitude,location,full_address
0,CE737959,BAA1QM02,2024-02-03 17:14:00+00,Open,1,25,Improper storage trash: res,2,NaN,Union Park,...,2 UNION PK ST,NaN,BOSTON,MA,02118,139372,42.341980,-71.070151,"(42.34197953680218, -71.07015050989436)",[2 Union Park St]
1,CE737958,LEHCWL02,2024-02-03 16:40:00+00,Open,1,25,Improper storage trash: res,46,NaN,Newton,...,447 SHAWMUT AV,C/O LAILA MAALOUF,BOSTON,MA,02118,102642,42.339440,-71.074631,"(42.3394395428274, -71.07463052618378)",[46 W Newton St]
2,CE737957,DUVKL802,2024-02-03 16:15:00+00,Open,1,25,Improper storage trash: res,294,296,Columbus,...,296 COLUMBUS AV,NaN,BOSTON,MA,02116,163593,42.346545,-71.073958,"(42.346545424034154, -71.07395778109957)",[294-296 Columbus Ave]
3,CE737956,2L5NDF16,2024-02-03 16:14:00+00,Open,3,100,Overfilling of barrel/dumpster,504,NaN,Dudley,...,520 DUDLEY ST,NaN,ROXBURY,MA,02119,226709,42.322790,-71.072701,"(42.322789542194485, -71.07270055853654)","[504 Dudley St, 2]"
4,CE737954,SZSDFX08,2024-02-03 16:03:00+00,Open,9a,250,Illegal dumping < 1 cubic yd,58,NaN,Mozart,...,23 WINDSOR DR,NaN,FOXBORO,MA,02035,99205,42.320800,-71.104401,"(42.32079958526445, -71.10440063469422)",[58 Mozart St]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683258,CE217197,CQV206,2000-01-01 18:54:00+00,Closed,1,25,Improper storage trash: res,73,NaN,Worcester,...,73 WORCESTER ST,NaN,BOSTON,MA,02118-3904,152520,42.339420,-71.077691,"(42.33941954680697, -71.07769053299197)",[73 Worcester St]
683259,CE217196,CP0Q06,2000-01-01 18:51:00+00,Closed,1,25,Improper storage trash: res,79,NaN,Worcester,...,79 WORCESTER ST,NaN,BOSTON,MA,02118,152532,42.339550,-71.077851,"(42.33954954765962, -71.07785053236364)",[79 Worcester St]
683260,CE217195,CKAU06,2000-01-01 18:44:00+00,Closed,1,25,Improper storage trash: res,92,NaN,Worcester,...,92 WORCESTER ST,C/O ROBERT WHALEN,BOSTON,MA,02118,152569,42.339640,-71.078541,"(42.33963954823288, -71.07854053395985)",[92 Worcester St]
683261,CE217194,CDE806,2000-01-01 18:33:00+00,Closed,1,25,Improper storage trash: res,17,NaN,Worcester,...,C/O URBAN MANAGEMENT,35 FAY ST,BOSTON,MA,02118,152404,42.338020,-71.076031,"(42.338019545006, -71.07603053224193)",[17 Worcester St]


In [27]:
# more misc cleaning and final check that all addresses are good
merged_code_vio.dropna(subset = ['ticket_no'], inplace = True)
print(merged_build_vio.isnull().sum())
print(merged_code_vio.isnull().sum())

case_no                 0
ap_case_defn_key        0
status_dttm             2
status                  0
code                    0
value               14783
description           225
violation_stno          0
violation_sthigh    10709
violation_street        0
violation_suffix      110
violation_city          1
violation_state         0
violation_zip           3
ward                    0
contact_addr1           5
contact_addr2       12119
contact_city            2
contact_state           2
contact_zip             6
sam_id                  0
latitude                0
longitude               0
location                0
geom_point_2249         0
geom_point_4326         0
full_address            0
dtype: int64
case_no                  0
ticket_no                0
status_dttm              0
status                   0
code                     0
value                    0
description              0
violation_stno           0
violation_sthigh    517974
violation_street         0
violation_suffi